# Gmail API
## Run query, save results as Excel, and send as attachment

This notebook will take in two input parameters: first, it will accept a semicolon-delimited (;) list of email addresses, and the query ID of the associated saved query in the workspace. Using these values, it loads the Databricks SDK, and reads the query text from the file and executes it as a dataframe operation.

With the resulting dataframe, it will then convert it to a Pandas dataframe, and save the result as Excel, using local node storage (for reasons: https://learn.microsoft.com/en-us/azure/databricks/files/write-data#where-should-i-write-temporary-files-on-azure-databricks)

Once saved, a new authentication request will be made using a username and pasword. For this example, you need to make sure that you:

1. Create a service account in your Google IAM and Admin page
2. Save your credentials file and upload the contents to a secret
3. Enable domain-wide delegation for the client ID

Secret scopes values need to be created as follows (feel free to change the scope name and/or secret names as needed).

```
Secret Scope name: dbx-mail-secerts

gmail-credentials-json: your credentials.json contents, in a secret
```

**Note**: If you plan on using the GMail API, this example assumes you've created a service account, downloaded and saved your credentals, approved IAM for the `https://www.googleapis.com/auth/gmail.send` scope, and enabled Domain delegation for the account.

In [0]:
%pip install openpyxl xlsxwriter google-api-python-client==1.7.8 google-auth-httplib2==0.0.3 google-auth-oauthlib==0.4.0

In [0]:
dbutils.library.restartPython()

In [0]:
dbutils.widgets.text("query_id", "", "Query ID")
dbutils.widgets.text("email_recipients", "", "Email Recipients (semicolon separated)")
dbutils.widgets.text("from_mailbox", "", "From Email Address")

In [0]:
from databricks.sdk import WorkspaceClient

w = WorkspaceClient()

In [0]:
requested_query = dbutils.widgets.get("query_id")
query = w.queries.get(requested_query)
query_obj = query.as_dict()
query_text = query_obj["query_text"]
query_name = query_obj["display_name"]

### DEBUG: What query actually ran?

In [0]:
print(query_text)

In [0]:
query_results_df = spark.sql(query_text)

In [0]:
query_results_pandas_df = query_results_df.toPandas()

In [0]:
import datetime
report_time = datetime.datetime.now()
report_time_formatted = report_time.strftime("%Y-%m-%d_%H:%M")
report_time_formatted


In [0]:
file_name = "/local_disk0/tmp/{0}_{1}.xlsx".format(query_name.replace(" ","_"), report_time_formatted)

In [0]:
import pandas as pd
writer = pd.ExcelWriter(file_name) 
query_results_pandas_df.to_excel(writer, sheet_name=query_name, index=False, na_rep='NaN')

for column in query_results_pandas_df:
    column_length = max(query_results_pandas_df[column].astype(str).map(len).max(), len(column))
    col_idx = query_results_pandas_df.columns.get_loc(column)
    writer.sheets[query_name].set_column(col_idx, col_idx, column_length)

writer.close()

### Set up email with GMail API and MIME

This notebook will now attempt to authenticate to the SMTP server via the secrets (see first cell) for server name and username and password.

In [0]:
import os.path
import base64
import mimetypes
import json
from email.message import EmailMessage
from google.oauth2.credentials import Credentials
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from google.oauth2 import service_account

SCOPES = ['https://www.googleapis.com/auth/gmail.send']
SERVICE_ACCOUNT_FILE = '/Volumes/main/default/initscripts/dublin-data-solutions-7ce1d0736e27.json'

credentials_json = json.loads(dbutils.widgets.get("credentials_json"))

credentials = service_account.Credentials.from_service_account_info(credentials_json, scopes=SCOPES, subject=dbutils.widgets.get("from_mailbox")

try:
    service = build("gmail", "v1", credentials=credentials)
    message = EmailMessage()

    message.set_content("This is automated draft mail")

    message["To"] = dbutils.widgets.get("email_recipients")
    message["From"] = subject=dbutils.widgets.get("from_mailbox")
    message["Subject"] = 'Please find your automated Databricks-geneated report attached. This report was generated on {0}'.format(report_time_formatted)

    # guessing the MIME type
    type_subtype, _ = mimetypes.guess_type(file_name)
    maintype, subtype = type_subtype.split("/")

    with open(file_name, "rb") as fp:
      attachment_data = fp.read()
    message.add_attachment(attachment_data, maintype, subtype, filename=os.path.basename(file_name))

    # encoded message
    encoded_message = base64.urlsafe_b64encode(message.as_bytes()).decode()

    create_message = {"raw": encoded_message}
    send_message = (
        service.users()
        .messages()
        .send(userId="me", body=create_message)
        .execute()
    )
    print(f'Message Id: {send_message["id"]}')
except HttpError as error:
    print(f"An error occurred: {error}")
    send_message = None